In [230]:
import fitz  # PyMuPDF
import re
from langdetect import detect
from typing import List,Tuple
import pandas as pd
from IPython.display import display

In [231]:
# Configuration
PDF_PATH = "../data/HSC26-Bangla1st-Paper.pdf"     #PDF Path
COLLECTION_NAME = "bangla_book"
CHUNK_SIZE = 300  # words
CHUNK_OVERLAP = 100  # words
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

print(f"PDF Path: {PDF_PATH}")
print(f"Chunk Size: {CHUNK_SIZE} words")
print(f"Overlap: {CHUNK_OVERLAP} words")
print(f"Model: {MODEL_NAME}")

PDF Path: ../data/HSC26-Bangla1st-Paper.pdf
Chunk Size: 300 words
Overlap: 100 words
Model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [232]:
# Text cleaning functions
def clean_extracted_text(text: str) -> str:
    """Clean and normalize extracted text"""
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove page numbers and headers/footers
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        # Skip very short lines that might be page numbers or artifacts
        if len(line) > 3 and not line.isdigit():
            cleaned_lines.append(line)
    
    text = ' '.join(cleaned_lines)
    
    # Normalize Bangla punctuation
    text = re.sub(r'[।]{2,}', '।', text)
    
    return text.strip()      

In [233]:
def detect_language_segments(text: str) -> List[Tuple[str, str]]:
    """Detect language segments in mixed Bangla-English text"""
    # Split by sentences using both Bangla and English sentence endings
    sentences = re.split(r'[।.!?]+', text)
    segments = []
    
    for sentence in sentences:
        sentence = sentence.strip()
        if len(sentence) < 5:  # Skip very short segments
            continue
            
        try:
            # Detect language
            lang = detect(sentence)
            # Map language codes
            if lang == 'bn':
                lang = 'bangla'
            elif lang == 'en':
                lang = 'english'
            else:
                lang = 'mixed'
                
            segments.append((sentence, lang))
        except:
            # If detection fails, mark as mixed
            segments.append((sentence, 'mixed'))
    
    return segments


In [234]:
# Extract text from PDF
print(f"Extracting text from: {PDF_PATH}")

doc = fitz.open(PDF_PATH)
pages_data = []

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text = page.get_text()
    
    # Clean extracted text
    text = clean_extracted_text(text)
    
    if text.strip():  # Only add non-empty pages
        pages_data.append({
            'page_number': page_num + 1,
            'text': text,
            'word_count': len(text.split())
        })

doc.close()

print(f"Extracted text from {len(pages_data)} pages")

Extracting text from: ../data/HSC26-Bangla1st-Paper.pdf
Extracted text from 49 pages


In [235]:
# Display sample page
if pages_data:
    sample_page = pages_data[0]
    print(f"\nSample from Page {sample_page['page_number']} ({sample_page['word_count']} words):")
    print(f"'{sample_page['text'][:200]}...'")


Sample from Page 1 (14 words):
'অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , অপরিরিতা আল ািয রিষয় িাাং া ১ম পত্র...'


In [236]:
# Analyze extracted content
total_words = sum(page['word_count'] for page in pages_data)
avg_words_per_page = total_words / len(pages_data) if pages_data else 0

print(f"Content Analysis:")
print(f"   Total Pages: {len(pages_data)}")
print(f"   Total Words: {total_words}")
print(f"   Average Words per Page: {avg_words_per_page:.1f}")

# Create a quick visualization
page_stats = pd.DataFrame(pages_data)
display(page_stats.head(10))

Content Analysis:
   Total Pages: 49
   Total Words: 7261
   Average Words per Page: 148.2


,page_number,text,word_count
0,1,"অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা , অপ...",14
1,2,১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক)...,202
2,3,শব্দার্ব ও টীকা েূ শব্দ শলব্দি অর্ব ও িযাখ্যা ...,271
3,4,শব্দার্ব ও টীকা েূ শব্দ শলব্দি অর্ব ও িযাখ্যা ...,210
4,5,শব্দার্ব ও টীকা েূ শব্দ শলব্দি অর্ব ও িযাখ্যা ...,72
5,6,েূ গ্ে আিআমািব্ সসাতািমাত্র।এিীব্নটানাদদকঘিযির...,112
6,7,আমািহর্িিকানপুকিকািককি।যসিুটিকতকজলকাতা আর্স াআ...,120
7,8,“মন্দন যহ! খাটিযসানা ব্কট!” র্ব্নুদাদািভাষাটাঅ...,143
8,9,মামার্ব্ব্াহ-ব্ার়্েকত ুর্ক াখুর্িহইকলননা।এককয...,121
9,10,এইব্জল াযেমকিমুখাযমাটাএকখানাব্ালা একটুচাপর্দ া...,104


In [237]:
pages_data[1]['text']


"১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন? ক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা ২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি- ক) প্রর্তপজি খ) প্রভাব্ গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি র্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও। র্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা তাি র্ব্ক ি উকদযাগ র্নকলও যেৌতুক র্নক ব্া়োব্ার়্ে কিাি কািকণ কনযাি র্পতা অপমার্নত যব্াধ ককি র্ব্ক ি আকলাচনা যভকে যদন। দীপু যমক টিি ির্ব্ যদকখ মুগ্ধ হকলও তাি চাচাকক র্কিুই ব্লকত পাকিনর্ন। ৩। েীপুি িািাি সলে ‘অপরিরিতা' গ্লেি দকান িরিলেি রে আলে? ক) হর্িকিি খ) মামাি গ) র্িক্ষককি ঘ) র্ব্নুি ৪। উক্ত িরিলে প্রাধানয দপলয়লে - i) যদৌিাত্ম ii) হীনম্মনযতা iii) যলাভ র্নকচি যকানটি ঠিক? ক। i ও ii খ। ii ও iii গ। i ও iii ঘ। i, ii ও iii ৫. অনুপলেি িয়স কত িেি? ক) পঁর্চি খ) িাব্বিি গ) সাতাি ঘ) আটাি প্রাক-মূলযা ন কতগুকলা প্রকেি সঠিক উিি র্দকত পািকল? SL Ans SL Ans SL Ans SL Ans SL Ans ১ খ ২ গ ৩ খ ৪ ক ৫ গ ✓র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক ওঠািফকলসমাকিপর্িচ স

Extracted text is severly corrupted and PyMUDF library fails to recognize the texts correctly. So, better approach is using ocr and extract using Pytesseract

In [238]:
# Import libraries
from pdf2image import convert_from_path
import cv2
import numpy as np
import re
import pandas as pd
from IPython.display import display
import pytesseract

In [239]:
TESS_LANG = "ben"
DPI = 300

In [240]:
# OCR-based text extraction functions
def preprocess_image_for_ocr(image):
    """Preprocess image for better OCR results"""
    # Convert PIL image to numpy array
    img = np.array(image)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Apply slight Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (1, 1), 0)
    
    # Increase contrast
    contrast = cv2.convertScaleAbs(blurred, alpha=1.2, beta=10)
    
    return contrast


In [241]:
def extract_text_tesseract(image: Image.Image) -> str:
    """Extract Bangla text using Tesseract"""
    return pytesseract.image_to_string(image, lang=TESS_LANG)

In [242]:
def clean_bangla_text(text: str) -> str:
    """Clean Bangla OCR output"""
    text = re.sub(r'[^\u0980-\u09FF\s।,!?]', '', text)  # Keep Bangla and punctuation
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [243]:
try:
    # Convert PDF pages to images
    images = convert_from_path(PDF_PATH, dpi=DPI) 
    print(f"Converted {len(images)} pages to images")
except Exception as e:
    print(f"Error converting PDF: {e}")
    raise

Converted 49 pages to images


In [244]:
# OCR all pages
pages_data = []
for i, image in enumerate(images):
    print(f"🔍 Processing page {i+1}/{len(images)}...", end=" ")
    try:
        raw_text = extract_text_tesseract(image)
        cleaned_text = clean_bangla_text(raw_text)
        if cleaned_text:
            pages_data.append({
                'page_number': i + 1,
                'text': cleaned_text,
                'word_count': len(cleaned_text.split()),
                'raw_text': raw_text[:300] 
            })
            print(f"✅ ({len(cleaned_text.split())} words)")
        else:
            print("⚠️ No text found")
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✅ Finished OCR on {len(pages_data)} pages")

🔍 Processing page 1/49... ✅ (13 words)
🔍 Processing page 2/49... ✅ (214 words)
🔍 Processing page 3/49... ✅ (163 words)
🔍 Processing page 4/49... ✅ (145 words)
🔍 Processing page 5/49... ✅ (42 words)
🔍 Processing page 6/49... ✅ (318 words)
🔍 Processing page 7/49... ✅ (294 words)
🔍 Processing page 8/49... ✅ (404 words)
🔍 Processing page 9/49... ✅ (264 words)
🔍 Processing page 10/49... ✅ (211 words)
🔍 Processing page 11/49... ✅ (225 words)
🔍 Processing page 12/49... ✅ (372 words)
🔍 Processing page 13/49... ✅ (291 words)
🔍 Processing page 14/49... ✅ (304 words)
🔍 Processing page 15/49... ✅ (347 words)
🔍 Processing page 16/49... ✅ (148 words)
🔍 Processing page 17/49... ✅ (207 words)
🔍 Processing page 18/49... ✅ (219 words)
🔍 Processing page 19/49... ✅ (229 words)
🔍 Processing page 20/49... ✅ (195 words)
🔍 Processing page 21/49... ✅ (326 words)
🔍 Processing page 22/49... ✅ (243 words)
🔍 Processing page 23/49... ✅ (324 words)
🔍 Processing page 24/49... ✅ (328 words)
🔍 Processing page 25/49... 

In [245]:
# Summary Stats
total_words = sum(page['word_count'] for page in pages_data)
avg_words = total_words / len(pages_data) if pages_data else 0
print(f"\n Summary:")
print(f"   Pages processed: {len(pages_data)}")
print(f"   Total words: {total_words}")
print(f"   Average words per page: {avg_words:.1f}")


 Summary:
   Pages processed: 49
   Total words: 11908
   Average words per page: 243.0


In [246]:
# Display sample page
if pages_data:
    sample = pages_data[0]
    print(f"\n Sample Page {sample['page_number']}:")
    print(f"Raw OCR: {sample['raw_text'][:200]}")
    print(f"Cleaned: {sample['text'][:200]}...")



 Sample Page 1:
Raw OCR: 10940759

অনলাইন ব্যাট”

হি
বাংলা * ইংরেজি * আইসিটি

অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,

কলকরো ৬৬ 76919

Cleaned: অনলাইন ব্যাট হি বাংলা ইংরেজি আইসিটি অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়, কলকরো ৬৬...


In [247]:
# Create DataFrame for analysis
df = pd.DataFrame([{
    'page_number': p['page_number'],
    'word_count': p['word_count'],
    'sample_text': p['text'][:100] + "..." if len(p['text']) > 100 else p['text']
} for p in pages_data])

In [248]:
# Display table
print("\n First 10 Pages:")
display(df.head(10))


 First 10 Pages:


,page_number,word_count,sample_text
0,1,13,অনলাইন ব্যাট হি বাংলা ইংরেজি আইসিটি অনলাইন ব্য...
1,2,214,লুল জআললাইন ব্যাচ ১? নিম্নবিত্ত ব্যক্তির হঠাৎ ...
2,3,163,লুল জআললাইন ব্যাচ ১? গল্পের কথক চরিত্র অনুপমের...
3,4,145,লুল জআললাইন ব্যাচ ১? বিধানকর্তা বা শাস্ত্রপ্রণ...
4,5,42,লন ৯ অনলাইন ব্যাচ মাটির খোলের দুপাশে চামড়া লা...
5,6,318,লুল জআললাইন ব্যাচ ১? মূল আলোচ্য বিষয় মূল গল্প...
6,7,294,আমার হরিশ কানপুরে কাজ করে। সে ছুটিতে কলিকাতায়...
7,8,404,মন্দ নয় হে! খাটি সোনা বটে! বিনুদাদার ভাষাটা অ...
8,9,264,লুল জআললাইন ব্যাচ ১? মামা বিবাহবাড়িতে ঢুকিয়া...
9,10,211,", এই বলিয়া যে মকরমুখা মোটা একখানা বালায় একটু..."


In [249]:
# Language Classification
bangla_pages = 0
english_pages = 0
mixed_pages = 0

for page in pages_data:
    text = page['text']
    bangla_chars = len(re.findall(r'[অ-ঔঋঌএঐওঔক-হৎড়ঢ়য়০-৯]', text))
    english_chars = len(re.findall(r'[a-zA-Z]', text))
    total_chars = len(text)

    if total_chars > 0:
        bangla_ratio = bangla_chars / total_chars
        english_ratio = english_chars / total_chars
        if bangla_ratio > 0.6:
            bangla_pages += 1
        elif english_ratio > 0.6:
            english_pages += 1
        else:
            mixed_pages += 1

print(f"\n Language Classification:")
print(f"   Primarily Bangla pages: {bangla_pages}")
print(f"   Primarily English pages: {english_pages}")
print(f"   Mixed content pages: {mixed_pages}")


 Language Classification:
   Primarily Bangla pages: 0
   Primarily English pages: 0
   Mixed content pages: 49


In [250]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
import os 
import faiss 


In [301]:
from dotenv import load_dotenv

load_dotenv()

# Access the variables
hf_token = os.getenv("HF_TOKEN")
cohere_token = os.getenv("cohere_api_key")

print("HuggingFace Token:", hf_token[:5] + "..." if hf_token else "Not found")
print("Cohere Token:", cohere_token[:5] + "..." if cohere_token else "Not found")


HuggingFace Token: hf_al...
Cohere Token: d2R8c...


In [252]:
chunks = []
for page in pages_data:
    words = page['text'].split()
    for i in range(0, len(words), CHUNK_SIZE - CHUNK_OVERLAP):
        chunk_words = words[i : i + CHUNK_SIZE]
        chunk_text = " ".join(chunk_words)
        chunks.append({
            'text': chunk_text,
            'page_number': page['page_number']
        })

In [253]:
len(chunks)

87

In [254]:

co = cohere.Client(api_key=cohere_token)
# Choose the multilingual model best for Bangla + English
EMBED_MODEL = "embed-multilingual-v3.0"

In [255]:
def embed_texts(texts, input_type="search_document"):
    response = co.embed(texts=texts, model=EMBED_MODEL, input_type=input_type)
    return np.array(response.embeddings) 

In [256]:
# Embedding chunk texts
chunk_texts = [c['text'] for c in chunks]
chunk_embeddings = embed_texts(chunk_texts, input_type="search_document")


In [257]:
print(f"Embedded {len(chunk_texts)} chunks with shape: {chunk_embeddings.shape}")

Embedded 87 chunks with shape: (87, 1024)


In [258]:
chunk_embeds = np.array(chunk_embeddings) 
print(chunk_embeds.shape)

(87, 1024)


In [259]:
dim = chunk_embeds.shape[1]
index = faiss.IndexFlatL2(dim) 
print(index.is_trained)
index.add(np.float32(chunk_embeds))  # Ensure embeddings are float32

True


In [260]:
def search(query, top_k=5):
    query_emb = embed_texts([query], input_type="search_document")[0]
    D, I = index.search(np.float32([query_emb]), top_k)
    texts_np = np.array(chunk_texts)
    results = pd.DataFrame(data ={
        'texts' : texts_np[I[0]],
        'distances': D[0]
        })
    print(f"Search results for query '{query}':")
    return results

In [261]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
results = search(query, top_k=5)
results 

Search results for query 'অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?':


,texts,distances
0,লুল জআললাইন ব্যাচ ১? ৩০। আমার মতো অক্ষম দুনিয়...,0.874470
1,জিজ্ঞাসা করল? ক অনুপম খ অনুপমের মা গ জেনারেল ঘ...,0.874944
2,লুল আনলাইন ব্যাচ ১? সৃজনশীল প্রশ্ন প্রশ্ন ১ কন...,0.890538
3,লুল জআললাইন ব্যাচ ১? ৬৯। কার সঙ্গে পঞ্চশরের বি...,0.895162
4,দেয়। চোখের সামনে অন্যায় দেখলে ইচ্ছা থাকা সত্...,0.915209


In [262]:
query = "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?"
results = search(query, top_k=5)
results 

Search results for query 'কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?':


,texts,distances
0,আনল পাঠ্যপুস্তকের প্রশ্ন বহুনির্বাচনী ১। অনুপম...,0.706990
1,যৌতুকলোভী চরিত্র। তিনি অনুপমের বিয়ের জন্য একট...,0.881034
2,লন ৯ অনলাইন ব্যাচ ২৪। অপরিচিতা গল্পে কোন দ্বীপ...,0.889718
3,ক পণের অঙ্ক সামান্য বলে খ মেয়ের শিক্ষা কম বলে...,0.906082
4,লুল জআললাইন ব্যাচ ১? গল্পের কথক চরিত্র অনুপমের...,0.918622


In [263]:
query = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
results = search(query, top_k=5)
results 

Search results for query 'বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?':


,texts,distances
0,বিনুদার গ অনুপমের ৫৩। একবার মামার কাছে কথাটা প...,0.493609
1,লন ৯ অনলাইন ব্যাচ ৩৯। এখানে জায়গা আছে উক্তিটি...,0.712034
2,ও ৫ অনুপমের বয়স কত বছর? ক পঁচিশ খ ছাবিবিশ গ স...,0.816628
3,মেয়ের জীবন বা ভবিষ্যৎ শঙ্কামুক্ত রাখার নিমিত্...,0.845232
4,মন্দ নয় হে! খাটি সোনা বটে! বিনুদাদার ভাষাটা অ...,0.860326


In [264]:
from rank_bm25 import BM25Okapi 
from sklearn.feature_extraction import _stop_words
import string 

In [265]:
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.split():
        token = token.strip(string.punctuation)
        if token and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token.lower())
    return tokenized_doc

In [266]:
tokenized_corpus = []
for passage in tqdm(chunk_texts, desc="Tokenizing corpus"):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

Tokenizing corpus: 100%|██████████| 87/87 [00:00<00:00, 24281.64it/s]


In [267]:
def keyword_search(query, top_k=5,num_candidates=15):
    print("Input Query:", query)
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{ 'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    print(f"Top {top_k} results for query '{query}':")
    for hit in bm25_hits[:top_k]:
        print(f"  - ID: {hit['corpus_id']}, Score: {hit['score']:.4f}, Text: {chunk_texts[hit['corpus_id']][:100]}...")

In [268]:
keyword_search("অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?", top_k=5, num_candidates=15)
keyword_search("কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?", top_k=5, num_candidates=15)
keyword_search("বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?", top_k=5, num_candidates=15)

Input Query: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
Top 5 results for query 'অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?':
  - ID: 61, Score: 5.7928, Text: গরুর গাড়ি গ মোটর গাড়ি ৯ ঘ ঘোড়ার গাড়ি ৩৮। অন্নপূর্ণার কোলে গজাননের ছোট ভাইটি এখানে ছোট ভাইটি বলতে...
  - ID: 60, Score: 4.9513, Text: লন ৯ অনলাইন ব্যাচ ২৪। অপরিচিতা গল্পে কোন দ্বীপের উল্লেখ আছে? ক আন্দামান দ্বীপ খ হাইকু দ্বীপ গ ক্যারি...
  - ID: 66, Score: 4.7661, Text: লুল জআললাইন ব্যাচ ১? ৬৯। কার সঙ্গে পঞ্চশরের বিরোধ নেই বলে অনুপমের মনে হলো? ক গজাননের খ কার্তিকের গ প...
  - ID: 57, Score: 3.6289, Text: ৬। অপরিচিতা গল্পে নায়কের বয়স কত বলা হয়েছে? ক ২৮ বছর খ ২৬ বছর গ ২৭ বছর ঘ ২৫ বছর ৭ তবু ইহার বিশেষ ম...
  - ID: 70, Score: 3.3198, Text: লন ৯ অনলাইন ব্যাচ ৯৭। অপরিচিতা গল্পে কন্যার পিতার পরিচয় ফুটিয়ে তুলতে বলা হয়েছে বয়স তার চল্লিশের ...
Input Query: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Top 5 results for query 'কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?':
  - ID: 62, Score: 13.5100, Text: লন ৯ অনলাইন ব্যাচ ৩৯। এখা

In [269]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
results = co.rerank(query=query, documents=chunk_texts, top_n=3, return_documents=True)
print(f"Rerank results for query '{query}':")
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score, result.document.text)

Rerank results for query 'অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?':
0 0.71185225 লুল জআললাইন ব্যাচ ১? ৬৯। কার সঙ্গে পঞ্চশরের বিরোধ নেই বলে অনুপমের মনে হলো? ক গজাননের খ কার্তিকের গ প্রজাপতির ঘ অন্নপূর্ণা ৭০। সুপুরুষ বটে কে? ক অনুপম খ হরিশ গ মামা ঘ শস্তুনাথ ৭১। চুল কাচা গোঁফ পাক ধরেছে কার? ক মামার খ শস্তুনাথের গ বিনুদাদার ঘ হরিশের ৭২। কল্যাণী কোন স্টেশন নেমে গেল? ক কোন্নগর খ কলিকাতা গ কানপুর ঘ হাওড়া ৭৩। ছোটবেলায় পণ্ডিত মশায় বিদ্রপ করত কেন? ক কুৎসিত এবং নিগ্ডণ হওয়ার কারণে খ কুৎসিত হয়ে গুণবান হওয়ার কারণে গ সুদর্শন এবং গুণবান হওয়ার কারণে ঘ সুদর্শন হয়েও নির্ভণ হওয়ার কারণে ৭৪। অনুপমকে বিবাহ আসর থেকে ফিরিয়ে দেবার কারণ কী? ক অনুপমের ব্যক্তিত্বহীনতার কারণে খ মামার হীনম্মন্যতার কারণে গ গয়না নিয়ে মনোমালিন্যের কারণে ঘ কনের বাবার আত্মগরিমার কারণে ৭৫। আমার পুরোপুরি বয়সই হলো না কথাটি দ্বারা কী বোঝানো হয়েছে? ক তরুণ বয়সী খ অপরিণত বয়সী গ অতি নির্ভতশীল ঘ চিন্তায় অপরিণত ৭৬। তামাকটুকু পর্যন্ত খাই না উক্তিটি দ্বারা কী বোঝানো হয়েছে? ক তামাক ক্ষতিকর খ তামাক অপছন্দ গ অতি ভালো মানুষ ঘ খাওয়ায় 

In [270]:
def keyword_and_rerank_search(query, top_k=5, num_candidates=15):
    print("Input Query:", query)

    # Step 1: Keyword Search using BM25
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{
        'corpus_id': idx,
        'score': bm25_scores[idx],
        'text': chunk_texts[idx]
    } for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    candidate_texts = [hit['text'] for hit in bm25_hits]

    # Step 2: Reranking using Cohere
    results = co.rerank(query=query, documents=candidate_texts, top_n=top_k, return_documents=True)

    # Step 3: Print results
    print(f"\nTop {top_k} reranked results for query '{query}':")
    for idx, result in enumerate(results.results):
        print(f"{idx + 1}. Relevance Score: {result.relevance_score:.4f}")
        print(f"   Text: {result.document.text[:150]}...\n")

    return [(result.document.text, result.relevance_score) for result in results.results]


In [271]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
top_docs = keyword_and_rerank_search(query, top_k=3, num_candidates=15)


Input Query: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?

Top 3 reranked results for query 'অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?':
1. Relevance Score: 0.7119
   Text: লুল জআললাইন ব্যাচ ১? ৬৯। কার সঙ্গে পঞ্চশরের বিরোধ নেই বলে অনুপমের মনে হলো? ক গজাননের খ কার্তিকের গ প্রজাপতির ঘ অন্নপূর্ণা ৭০। সুপুরুষ বটে কে? ক অনুপম ...

2. Relevance Score: 0.5646
   Text: লুল জআললাইন ব্যাচ ১? ৩০। আমার মতো অক্ষম দুনিয়ায় নাই। অনুপমের এই উক্তির মধ্য দিয়ে কী প্রকাশ পেয়েছে? কু বো২২ অনুশোচনা অসহায়ত্ব ক্ষোভ নিচের কোনটি সঠ...

3. Relevance Score: 0.3892
   Text: লন ৯ অনলাইন ব্যাচ ৯ ৫। মামার বাহিরের যাত্রাপথের সীমানা কতদূর? ক আন্দামান পর্যন্ত খ কোন্নগর পর্যন্ত গ কানপুর পর্যন্ত ঘ হাওড়া পর্যন্ত ৫৬। বিবাহের কতদিন...



In [299]:
from langchain.vectorstores import FAISS
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.chat_models import ChatCohere
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import Document

In [289]:
# Use MPS device for HuggingFace embeddings (Apple Silicon GPU)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

device = "mps"  # Apple Silicon GPU

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": device}
)

docs = [Document(page_content=chunk['text']) for chunk in chunks]
faiss_store = FAISS.from_documents(docs, embedding=embedding_model)
faiss_store.save_local("../faiss_index")
print("FAISS index saved to 'faiss_index/'")

FAISS index saved to 'faiss_index/'


In [290]:
#Faiss load local
faiss_store = FAISS.load_local(
    "../faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


In [318]:
template = """প্রসঙ্গ:
নিচে একটি অজানা ধরনের তথ্যসূত্র (context) প্রদান করা হয়েছে। এটি একটি গল্প, প্রশ্নোত্তর, বহু নির্বাচনী প্রশ্ন, সাধারণ জ্ঞান, আলোচনা অথবা অন্য যেকোনো ধরণের লেখা হতে পারে। 
এই তথ্যসূত্রের ভিতরে বা শেষে কিছু প্রশ্নের সঠিক উত্তর সরাসরি বা পরোক্ষভাবে উপস্থিত থাকতে পারে — যেমন শেষে সঠিক উত্তর তালিকাবদ্ধ থাকতে পারে (যেমন: "Correct Answers", "Answer Key" ইত্যাদি), অথবা উত্তরগুলো লেখার ভিতরে ছড়িয়ে থাকতে পারে।

আপনার কাজ:
- পুরো তথ্যসূত্র (context) মনোযোগ দিয়ে পড়ুন।
- প্রদত্ত প্রশ্নের সঠিক উত্তর তথ্যসূত্র থেকে খুঁজে বের করুন।
- যদি সঠিক উত্তর context-এর শেষে আলাদাভাবে তালিকাভুক্ত থাকে, তাহলে সেখান থেকেও মিলিয়ে দেখুন।
- সঠিক উত্তরটি সংক্ষিপ্তভাবে এবং স্পষ্টভাবে এক লাইনে প্রদান করুন।

উত্তর লেখার নিয়ম:
- আপনার উত্তর অবশ্যই তথ্যসূত্রের উপর ভিত্তি করে হতে হবে।
- অপ্রাসঙ্গিক ব্যাখ্যা বা অতিরিক্ত কিছু যুক্ত করবেন না।
- প্রশ্ন যেভাবেই হোক, আপনি কেবলমাত্র সংশ্লিষ্ট ও সঠিক উত্তরটি এক লাইনে দিন।

{context}

প্রশ্ন: {question}
উত্তর: এক লাইনে সঠিক উত্তর দিন।"""
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)


In [319]:
memory = ConversationBufferWindowMemory(k=2, return_messages=True)

In [329]:
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_openai = ChatOpenAI(
    openai_api_key=openai_api_key,
    model="gpt-4o",       
    streaming=True       
)

In [328]:
def rag_chat_with_memory(query: str, llm=llm):
    # Save user question into memory
    memory.chat_memory.add_user_message(query)

    # Step 1: FAISS retrieval of top-10 candidate docs
    retriever = faiss_store.as_retriever(search_kwargs={"k": 10})
    candidate_docs = retriever.get_relevant_documents(query)
    
    # Extract texts and indices from FAISS results
    candidate_texts = [doc.page_content for doc in candidate_docs]
    
    # Step 2: Apply BM25 on the FAISS-retrieved documents
    # Tokenize query for BM25
    tokenized_query = bm25_tokenizer(query)
    
    # Create local BM25 index for the candidate texts
    tokenized_candidates = [bm25_tokenizer(text) for text in candidate_texts]
    local_bm25 = BM25Okapi(tokenized_candidates)
    
    # Get BM25 scores for candidate documents
    bm25_scores = local_bm25.get_scores(tokenized_query)
    
    # Combine texts with their BM25 scores
    bm25_results = [{
        'text': text,
        'score': score
    } for text, score in zip(candidate_texts, bm25_scores)]
    
    # Sort by BM25 scores (descending)
    bm25_results = sorted(bm25_results, key=lambda x: x['score'], reverse=True)
    
    # Step 3: Rerank BM25-ordered texts with Cohere to get top-3
    rerank_res = co.rerank(
        query=query,
        documents=[res['text'] for res in bm25_results],
        top_n=3,
        return_documents=True
    )
    
    # Step 4: Extract top reranked texts
    top_texts = [res.document.text for res in rerank_res.results]
    
    # Step 5: Build context from reranked texts
    context = "\n".join(top_texts)

    # Step 6: Generate answer using prompt template
    prompt_str = prompt_template.format(context=context, question=query)
    answer = llm.invoke([HumanMessage(content=prompt_str)]).content.strip()
    
    # Save answer into memory
    memory.chat_memory.add_ai_message(answer)
    
    # Print and return
    print("User:", query)
    print("Answer:", answer)
    return answer

In [330]:
from langchain.chat_models import ChatCohere

llm_command_r = ChatCohere(cohere_api_key=cohere_token, model="command-r", streaming=True)


In [331]:
rag_chat_with_memory(query="বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?", llm=llm_openai) 

User: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
Answer: তথ্যসূত্রে কল্যাণীর প্রকৃত বয়স সরাসরি উল্লিখিত নেই।


'তথ্যসূত্রে কল্যাণীর প্রকৃত বয়স সরাসরি উল্লিখিত নেই।'

In [342]:
rag_chat_with_memory(query="বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?", llm=llm_command_r) 

KeyboardInterrupt: 

In [333]:
memory.chat_memory.messages[-4:]  # Show last 2 messages in memory

[AIMessage(content='প্রশ্নের উত্তরের জন্য প্রদত্ত তথ্যসূত্রে সরাসরি কোনো উল্লেখ নেই।'),
 HumanMessage(content='বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?'),
 AIMessage(content='তথ্যসূত্রে কল্যাণীর প্রকৃত বয়স সরাসরি উল্লিখিত নেই।'),
 HumanMessage(content='বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?')]

In [341]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline

# Load Hugging Face model and tokenizer
model_id = "Qwen/Qwen1.5-1.8B-Chat"  
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Create a text generation pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

# Wrap in LangChain-compatible LLM
llm_qwen = HuggingFacePipeline(pipeline=pipe)


{"timestamp":"2025-07-25T18:06:52.101285Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(SendRequest, hyper::Error(IncompleteMessage)) }). Retrying..."},"filename":"/Users/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-25T18:06:52.101412Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 1.27884763s before the next attempt"},"filename":"/Users/runner/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:
rag_chat_with_memory("অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?", llm=llm_qwen)